In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/ece176_final_project-main')

!pip3 install -r requirements.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Set memory allocation configuration
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
import torch
import numpy as np
import gymnasium as gym
import ale_py
import matplotlib.pyplot as plt
import os

# Import your modules
from utils.preprocessor import DQNPreprocessor
from models.base import DQN
from models.dqn2015 import DQN2
from utils.replayBuffer import SequentialGPUReplayBuffer
from agents.base import DQNAgent
from utils.visualizer import DQNVisualizer

In [ ]:
def train_breakout_dqn(num_frames=10000000,
                    memory_size=500000,          # Smaller replay buffer
                    batch_size=32,
                    gamma=0.99,
                    eps_start=1.0,
                    eps_end=0.1,
                    eps_decay=250000,             # Very fast epsilon decay
                    target_update=5000,          # Update target network frequently
                    learning_rate=0.0025,       # Slightly higher learning rate
                    update_freq=4,
                    replay_start_size=25000,     # Start training after just 1000 frames
                    no_op_max=5,
                    eval_interval=250000,
                    save_interval=500000):
    """
    Short test training of a DQN agent on Breakout.
    """
    # 1. Environment setup
    env_name = "CarnivalDeterministic-v4"
    env = gym.make(env_name)

    # Set device
    device = None
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
    print(f"Using device: {device}")

    # 2. Create the agent with minimized hyperparameters
    agent = DQNAgent(
        env=env,
        replayBufferClass=SequentialGPUReplayBuffer,
        frameShape=(4, 84, 84),
        QNetwork=DQN2,
        PreprocessorClass=DQNPreprocessor,
        device=device,
        memory_size=memory_size,          # Smaller replay buffer
        batch_size=batch_size,
        gamma=gamma,
        eps_start=eps_start,
        eps_end=eps_end,
        eps_decay=eps_decay,             # Very fast epsilon decay
        target_update=target_update,          # Update target network frequently
        learning_rate=learning_rate,       # Slightly higher learning rate
        update_freq=update_freq,
        replay_start_size=replay_start_size,     # Start training after just 1000 frames
        no_op_max=no_op_max,                # Fewer no-ops at start of episode
        eval_interval=eval_interval,
        save_interval=save_interval
    )

    # 3. Train for a small number of frames
    print(f"Starting quick test training for {num_frames} frames...")

    episode_rewards, eval_rewards = agent.train(num_frames=num_frames)
    print("Training completed!")

    # 4. Plot training episode rewards
    os.makedirs("train_runs", exist_ok=True)
    plt.figure(figsize=(12,6))
    plt.plot(episode_rewards, label="Episode Reward")
    plt.title("Training Rewards over Episodes")
    plt.xlabel("Episode")
    plt.ylabel("Reward")
    plt.legend()
    plt.grid(True)
    plt.savefig("train_runs/breakout_dqn_test_training_rewards.png")
    plt.close()

    # 5. Quick evaluation - just 3 episodes
    eval_score = agent.evaluate(num_episodes=3)
    print(f"Evaluation over 3 episodes: {eval_score:.2f} average reward")

    # 6. Clean up
    env.close()

Test training with minimial parameters

In [ ]:
train_breakout_dqn(num_frames=2000,
        memory_size=10000,          # Smaller replay buffer
        batch_size=32,
        gamma=0.99,
        eps_start=1.0,
        eps_end=0.1,
        eps_decay=2000,             # Very fast epsilon decay
        target_update=500,          # Update target network frequently
        learning_rate=0.0025,       # Slightly higher learning rate
        update_freq=4,
        replay_start_size=1000,     # Start training after just 1000 frames
        no_op_max=5)

Using device: cuda
Allocating sequential replay buffer with capacity 10000 on cuda...
Allocating sequential states tensor...
Allocating actions tensor...
Allocating rewards tensor...
Allocating dones tensor...
Sequential buffer allocation complete!
Estimated GPU memory usage: 0.26 GiB
Starting quick test training for 2000 frames...

Filling replay memory with 1000 frames of random experience...


Filling Replay Memory: 100%|██████████| 1000/1000 [00:01<00:00, 875.32it/s]


Replay memory filled!


Training:  25%|██▌       | 507/2000 [00:01<00:03, 377.55it/s]

Episode 1 completed | Reward: 660.00 | Avg Reward: 660.00 | Frames: 433 | Epsilon: 0.7835


...

Episode 3 completed | Reward: 480.00 | Avg Reward: 540.00 | Frames: 1269 | Epsilon: 0.3655


Training:  86%|████████▋ | 1729/2000 [00:05<00:00, 341.09it/s]

Episode 4 completed | Reward: 380.00 | Avg Reward: 500.00 | Frames: 1683 | Epsilon: 0.1585


Training: 100%|██████████| 2000/2000 [00:05<00:00, 338.54it/s]


Episode 5 completed | Reward: 340.00 | Avg Reward: 468.00 | Frames: 2000 | Epsilon: 0.1000

Training completed!
Model saved to weights/CarnivalDeterministic-v4_dqn_final.pth
Training completed!
Evaluation over 3 episodes: 253.33 average reward


Full training for 10 million frames

In [ ]:
train_breakout_dqn(num_frames=10000000,
                    memory_size=1000000,
                    batch_size=32,
                    gamma=0.99,
                    eps_start=1.0,
                    eps_end=0.1,
                    eps_decay=1000000,
                    target_update=5000,
                    learning_rate=0.00025,
                    update_freq=4,
                    replay_start_size=50000,
                    no_op_max=30,
                    eval_interval=250000,
                    save_interval=500000)

Using device: cuda
Allocating sequential replay buffer with capacity 1000000 on cuda...
Allocating sequential states tensor...
Allocating actions tensor...
Allocating rewards tensor...
Allocating dones tensor...
Sequential buffer allocation complete!
Estimated GPU memory usage: 26.29 GiB
Starting quick test training for 10000000 frames...

Filling replay memory with 50000 frames of random experience...


Filling Replay Memory: 100%|██████████| 50000/50000 [00:52<00:00, 961.07it/s]


Replay memory filled!


Training:   0%|          | 481/10000000 [00:01<8:31:34, 325.77it/s]

Episode 1 completed | Reward: 1000.00 | Avg Reward: 1000.00 | Frames: 427 | Epsilon: 0.9996


...

Training:  21%|██        | 2100177/10000000 [9:24:13<46:48:52, 46.87it/s]

Episode 3852 completed | Reward: 1700.00 | Avg Reward: 2223.30 | Frames: 2100168 | Epsilon: 0.1000


Training:  21%|██        | 2100709/10000000 [9:24:24<46:49:14, 46.86it/s]

Episode 3853 completed | Reward: 1260.00 | Avg Reward: 2210.30 | Frames: 2100701 | Epsilon: 0.1000


Training:  21%|██        | 2100742/10000000 [9:24:24<45:27:01, 48.28it/s]